In [ ]:
!pip install keras-core --upgrade
!pip install -q keras-nlp --upgrade
!pip install datasets
!pip install accelerate -U
!pip install -U transformers[pytorch]
!pip install beautifulsoup4


In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

if tf.test.gpu_device_name():
    print("GPU is available.")
else:
    print("GPU is not available.")

TensorFlow version: 2.15.0
GPU is available.


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_core as keras
import keras_nlp
import matplotlib.pyplot as plt

In [ ]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [ ]:
df_train_copy = df_train.copy()
df_train_copy.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
import re
from bs4 import BeautifulSoup


joined = df_train.copy()
joined["text"] = joined['text'].str.lower()
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub('', text)


joined['text'] = joined['text'].apply(lambda x: remove_html_tags(x))
joined['text'] = joined['text'].apply(lambda x: remove_urls(x))

print("Html tags removed")

<ipython-input-81-81e50666eb41>:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


Html tags removed


In [ ]:
def remove_mentions(text):
    return re.sub(r'@[A-Za-z0-9_]+', '', text)

def remove_hashtags(text):
    return re.sub(r'#[A-Za-z0-9_]+', '', text)

joined['text'] = joined['text'].apply(lambda x: remove_mentions(x))
joined['text'] = joined['text'].apply(lambda x: remove_hashtags(x))
print("Hastags removed")

Hastags removed


In [ ]:
joined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [ ]:
df_train = joined.iloc[:, :5].copy()

In [ ]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this may allah fo...,1
1,4,NaN,NaN,forest fire near la ronge sask. canada,1
2,5,NaN,NaN,all residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive evacuation orders in ca...",1
4,7,NaN,NaN,just got sent this photo from ruby as smoke f...,1


In [ ]:
from datasets import Dataset
train = Dataset.from_pandas(df=df_train[["text", "target"]])

train = train.class_encode_column("target")
train

Stringifying the column:   0%|          | 0/7613 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/7613 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'target'],
    num_rows: 7613
})

In [ ]:
test =Dataset.from_pandas(df=df_test[["text"]])
test

Dataset({
    features: ['text'],
    num_rows: 3263
})

In [ ]:
train_split = train.train_test_split(
    test_size = 0.2,
    seed=42
)
train_split

DatasetDict({
    train: Dataset({
        features: ['text', 'target'],
        num_rows: 6090
    })
    test: Dataset({
        features: ['text', 'target'],
        num_rows: 1523
    })
})

In [ ]:
from transformers import AutoTokenizer

model_ckpt = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
train_split['train'][300]['text']

'Russian    fur hat (xl61-62) with soviet badge LINK:\n '

In [ ]:
sample_text = train_split['train'][300]['text']
print(sample_text)

Russian    fur hat (xl61-62) with soviet badge LINK:
 


In [ ]:
sample_text_tokenized = tokenizer(sample_text)
print(sample_text_tokenized)

{'input_ids': [0, 14836, 1437, 1437, 1437, 15503, 3988, 36, 1178, 462, 5606, 12, 5379, 43, 19, 98, 705, 5810, 22394, 25838, 35, 50118, 1437, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
len(sample_text_tokenized)

2

In [ ]:
def tokenize(batch):
  return tokenizer(batch["text"], truncation= True, max_length=512)

dataset_tokenized= train_split.map(tokenize, batched=True)

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

In [ ]:
dataset_tokenized

DatasetDict({
    train: Dataset({
        features: ['text', 'target', 'input_ids', 'attention_mask'],
        num_rows: 6090
    })
    test: Dataset({
        features: ['text', 'target', 'input_ids', 'attention_mask'],
        num_rows: 1523
    })
})

In [ ]:
test_dataset_tokenized= test.map(tokenize, batched=True)
test_dataset_tokenized

Map:   0%|          | 0/3263 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 3263
})

In [ ]:
dataset_tokenized = dataset_tokenized.rename_column("target", "labels")

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

num_labels=2

model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt,
    num_labels=num_labels,
    ignore_mismatched_sizes =True
)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
import accelerate

accelerate.__version__

'0.26.1'

In [ ]:
if torch.cuda.is_available():
    # Set the device to GPU
    device = torch.device("cuda")
    print("Using GPU for training.")
else:
    # Set the device to CPU
    device = torch.device("cpu")
    print("GPU not available. Using CPU for training.")

Using GPU for training.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "roberta-disaster-tweets",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True,
    disable_tqdm=False,
    push_to_hub=False,
    report_to="none",
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset_tokenized["train"],
    eval_dataset=dataset_tokenized["test"],
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.437000,0.396329,0.834537,0.832750
2,0.334000,0.406650,0.828628,0.825698
3,0.264600,0.511398,0.816809,0.816718
4,0.206600,0.581086,0.818779,0.817067
5,0.162500,0.605605,0.818779,0.818082


Checkpoint destination directory roberta-disaster-tweets/checkpoint-191 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory roberta-disaster-tweets/checkpoint-382 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory roberta-disaster-tweets/checkpoint-573 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory roberta-disaster-tweets/checkpoint-764 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory roberta-disaster-tweets/checkpoint-955 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=955, training_loss=0.28091665138124794, metrics={'train_runtime': 176.4589, 'train_samples_per_second': 172.561, 'train_steps_per_second': 5.412, 'total_flos': 636755476187760.0, 'train_loss': 0.28091665138124794, 'epoch': 5.0})

In [ ]:
results = trainer.evaluate()

print(results)
print()
print(f"accuracy: {results['eval_accuracy']}")
print(f"f1: {results['eval_f1']}")

{'eval_loss': 0.3963291645050049, 'eval_accuracy': 0.8345370978332239, 'eval_f1': 0.832749535972017, 'eval_runtime': 3.0091, 'eval_samples_per_second': 506.124, 'eval_steps_per_second': 63.473, 'epoch': 5.0}

accuracy: 0.8345370978332239
f1: 0.832749535972017


In [ ]:
test_predictions = trainer.predict(test_dataset_tokenized)
test_predictions

PredictionOutput(predictions=array([[-1.1025391,  1.4267578],
       [-1.5234375,  1.7587891],
       [-1.0419922,  1.1113281],
       ...,
       [-2.0703125,  2.1621094],
       [-0.9453125,  1.1318359],
       [-1.5107422,  1.7216797]], dtype=float32), label_ids=None, metrics={'test_runtime': 7.9105, 'test_samples_per_second': 412.488, 'test_steps_per_second': 51.577})

In [ ]:
predictions = np.argmax(test_predictions.predictions, axis=1)
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
df_test['target'] = predictions
Final_result = df_test[['id', 'target']]
Final_result.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [ ]:
Final_result.to_csv('Final_result.csv', index=False)